In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     # only use GPU memory that we need, not allocate all the GPU memory
#     tf.config.experimental.set_memory_growth(gpus[0], enable=True)

# Set CPU as available physical device (for BERT)
physical_devices = tf.config.list_physical_devices('GPU')
try:
  # Disable first GPU
  tf.config.set_visible_devices(physical_devices[1:], 'GPU')
  logical_devices = tf.config.list_logical_devices('GPU')
  # Logical device was not created for first GPU
  assert len(logical_devices) == len(physical_devices) - 1
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

#https://predictivehacks.com/?all-tips=how-to-get-bert-embeddings-with-tensorflow-hub
#https://www.analyticsvidhya.com/blog/2021/09/performing-email-spam-detection-using-bert-in-python/
#https://www.analyticsvidhya.com/blog/2020/10/simple-text-multi-classification-task-using-keras-bert/
#https://medium.com/analytics-vidhya/understanding-bert-usage-31af2042be9e

In [4]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [5]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
df = pd.read_csv('spam.csv', encoding='Latin-1')

In [7]:
df = df.loc[:,['v1','v2']]
df.columns = ['target','text']
df

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [8]:
df['target_encoded'] = df['target'].map({'ham':0,'spam':1})

In [9]:
from sklearn.model_selection import train_test_split

xtrain, xval, ytrain, yval = train_test_split(df['text'], df["target_encoded"], test_size=0.2, stratify=df["target_encoded"])

In [10]:
# downloading preprocessing files and model
import tensorflow_hub as hub
import tensorflow_text as text  # Imports TF ops for preprocessing

bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

2022-11-11 10:16:02.405062: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [11]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'Inputs')
preprocessed_text = bert_preprocessor(text_input)
embeed = bert_encoder(preprocessed_text)
layer1 = tf.keras.layers.Dense(64, activation='relu')(embeed['pooled_output']),
layer2 = tf.keras.layers.Dropout(0.1)(layer1)
layer3 = tf.keras.layers.Dense(32, activation='relu')(layer2)
layer4 = tf.keras.layers.Dropout(0.1)(layer3)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'Dense')(layer4)

# creating final model
model = tf.keras.Model(inputs = [text_input], outputs = [outputs])

In [12]:
import keras.backend as K

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [13]:
model.compile(optimizer ='adam',
               loss = 'binary_crossentropy',
               metrics = [f1,tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
                            tf.keras.metrics.Precision(name = 'precision'),
                            tf.keras.metrics.Recall(name = 'recall')
           ])

In [14]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [15]:
history = model.fit(xtrain, ytrain, epochs = 5, verbose=True, callbacks=[tensorboard_callback])

# with tf.device('/cpu:0'): ?

Epoch 1/5
140/140 [==============================] - 332s 2s/step - loss: 0.4151 - f1: 0.0018 - accuracy: 0.8599 - precision: 0.1714 - recall: 0.0100
Epoch 2/5
 12/140 [=>............................] - ETA: 4:31 - loss: 0.3985 - f1: 0.0000e+00 - accuracy: 0.8698 - precision: 0.0000e+00 - recall: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
model.evaluate(xval,yval)

35/35 [==============================] - 75s 2s/step - loss: 0.1814 - f1: 0.5857 - accuracy: 0.9291 - precision: 0.9268 - recall: 0.5101


[0.18141554296016693,
 0.5857062935829163,
 0.9291479587554932,
 0.9268292784690857,
 0.5100671052932739]

In [ ]:

np.save('history1.npy',history.history)
history1=np.load('history1.npy', allow_pickle='TRUE').item()


In [ ]:
# downloading preprocessing files and model
import tensorflow_hub as hub
import tensorflow_text as text  # Imports TF ops for preprocessing

# Define the model
BERT_MODEL = "https://tfhub.dev/google/experts/bert/wiki_books/2"
# Choose the preprocessing that must match the model
PREPROCESS_MODEL = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

preprocess = hub.load(PREPROCESS_MODEL)
bert = hub.load(BERT_MODEL)

2022-11-09 16:28:05.600228: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Inputs (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['Inputs[0][0]']                 
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
def text_to_emb(input_text):
  input_text_lst = [input_text]
  inputs = preprocess(input_text_lst)
  outputs = bert(inputs)
  return np.array((outputs['pooled_output']))

In [ ]:
print(text_to_emb('This is a sample sentence'))
  

[[ 0.6730153   0.16980024  0.0517062   0.49646884  0.5704644   0.3031674
  -0.07520075 -0.9905823   0.9418097   0.3562777  -0.9015574  -0.9523854
  -0.2330077   0.3878442  -0.39498916 -0.21855661  0.23766173  0.987274
  -0.10497741  0.7786637  -0.29289722  0.71026987  0.3471179  -0.5728613
   0.9419618   0.37325022 -0.5141686   0.85436046  0.20006509 -0.31995782
   0.1932226  -0.26115093 -0.28056735  0.17820473 -0.01893848  0.53536505
  -0.60382813  0.7433764  -0.9028417   0.05744893  0.47359225  0.2269099
   0.22758965 -0.18882677  0.74394166 -0.16068767  0.27666396  0.657109
   0.60050666 -0.38498506  0.02086009  0.905419    0.61438924 -0.64821756
   0.20391536 -0.65669554  0.91290754  0.05876156 -0.43066576 -0.85168594
  -0.6492433   0.5278645  -0.72371054 -0.559621   -0.08929721 -0.12537009
   0.2024772   0.80765194  0.82415664  0.40183684 -0.8625045   0.7244723
  -0.7604728   0.8433044  -0.03106003  0.8740302  -0.64227927 -0.62824893
   0.89689255  0.37437546  0.72798795 -0.815550